In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv


In [2]:
!pip install transformers

In [3]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

train_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
test_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

train_df.head()

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


**Preprocess the Data
I'll format the input for the T5 model by combining the original_text and rewritten_text to create a single input prompt for the model to learn from.**

In [4]:
def preprocess_data(original_text, rewritten_text):
    return f"Original: {original_text} Rewritten: {rewritten_text} What was the rewrite prompt?"
train_df['input_text'] = train_df.apply(lambda row: preprocess_data(row['original_text'], row['rewritten_text']), axis=1)
train_df[['input_text', 'rewrite_prompt']].head()

,input_text,rewrite_prompt
0,Original: The competition dataset comprises te...,"Convert this into a sea shanty: """"""The competi..."


**loading the pre-trained T5-base model and tokenizer from Hugging Face. The T5-base model is small enough for fine-tuning in a Kaggle environment.**

In [5]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Tokenize the Input and Labels**

In [6]:
train_encodings = tokenizer(train_df['input_text'].tolist(), max_length=512, truncation=True, padding=True, return_tensors="pt")
labels = tokenizer(train_df['rewrite_prompt'].tolist(), max_length=128, truncation=True, padding=True, return_tensors="pt")

**Fine-Tune the T5 Model**

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

epochs = 3 

for epoch in range(epochs):
    model.train()
    
    input_ids = train_encodings['input_ids'].to(device)
    attention_mask = train_encodings['attention_mask'].to(device)
    labels_ids = labels['input_ids'].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_ids)
    
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/3, Loss: 1.1603163480758667
Epoch 2/3, Loss: 1.0290485620498657
Epoch 3/3, Loss: 0.8679758906364441


**Preprocess the Test Data**

In [8]:
test_df['input_text'] = test_df.apply(lambda row: preprocess_data(row['original_text'], row['rewritten_text']), axis=1)

test_encodings = tokenizer(test_df['input_text'].tolist(), max_length=512, truncation=True, padding=True, return_tensors="pt")

**Generate Predictions for the Test Set**

In [9]:
model.eval()
with torch.no_grad():
    input_ids = test_encodings['input_ids'].to(device)
    attention_mask = test_encodings['attention_mask'].to(device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
predicted_prompts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
test_df['rewrite_prompt'] = predicted_prompts
test_df[['id', 'rewrite_prompt']].head()

,id,rewrite_prompt
0,-1,the competition dataset comprises text passage...


In [10]:
test_df[['id', 'rewrite_prompt']].to_csv('submission.csv', index=False)
print("Submission file created successfully!")

Submission file created successfully!
